In [1]:
import os
import sys
import re
import numpy as np
import pickle
from scipy.optimize import fmin
import multiprocessing as mp
import functools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# statmech library setup
sys.path.append('../../statmechlib')
from statmechlib.preprocessing import select_nodes, to_param_dict, to_param_list, insert_zero_params, rescale_manybody_params
from statmechlib.preprocessing import Trajectory, scale_configuration, pair_dist_cutoff, find_index
from statmechlib.forcefields import sd2_loss, utot_EAM_per_atom, utot_EAM_per_box, udif_print, u_core
from statmechlib.preprocessing import universal_eos, downselect
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM_per_atom, get_stats_EAM_per_box
from statmechlib.read_write import params_to_pickle

In [4]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [5]:
# stats_file = 'stats_marinica' # 'stats_samples'
# target_file = 'target_marinica' # 'target_samples'
params_file = 'marinica_params'
# stats_file = 'stats_manyknots' # 'stats_samples'
# target_file = 'target_manyknots' # 'target_samples'
# stats_file = 'stats_manymd' # 'stats_samples'
# target_file = 'target_manymd' # 'target_samples'
#stats_file = 'stats_manyall' # 'stats_samples'
#target_file = 'target_manyall' # 'target_samples'
stats_file = 'stats_all_samples' # 'stats_samples'
target_file = 'target_all_samples' # 'target_samples'
# stats_file = 'stats_everything' # 'stats_samples'
# target_file = 'target_everything' # 'target_samples'
stats_md_file = 'stats_onlymd' # 'stats_samples'
target_md_file = 'target_onlymd' # 'stats_samples'

In [6]:
with open(os.path.join(working, stats_file+'.pickle'), 'rb') as fi:
    stats_all = pickle.load(fi)

with open(os.path.join(working, target_file+'.pickle'), 'rb') as fi:
    targets = pickle.load(fi)
    
with open(os.path.join(working, stats_md_file+'.pickle'), 'rb') as fi:
    stats_md = pickle.load(fi)

with open(os.path.join(working, target_md_file+'.pickle'), 'rb') as fi:
    targets_md = pickle.load(fi)

with open(os.path.join(working, params_file + '.pickle'), 'rb') as fi:
    param_list = pickle.load(fi)  

In [7]:
print(stats_all['hyperparams']['edens'])

[2.02, 2.0600000000000001, 2.1000000000000001, 2.1400000000000001, 2.1800000000000002, 2.2200000000000002, 2.2599999999999998, 2.2999999999999998, 2.3399999999999999, 2.3799999999999999, 2.4199999999999999, 2.45, 2.46, 2.5, 2.54, 2.5648975, 2.5800000000000001, 2.6200000000000001, 2.629795, 2.6600000000000001, 2.6946925, 2.7000000000000002, 2.7400000000000002, 2.7799999999999998, 2.8199999999999998, 2.8599999999999999, 2.8663175, 2.8999999999999999, 2.9399999999999999, 2.973045, 2.98, 3.02, 3.0600000000000001, 3.0797725, 3.1000000000000001, 3.1400000000000001, 3.1800000000000002, 3.2200000000000002, 3.2599999999999998, 3.2999999999999998, 3.3399999999999999, 3.3799999999999999, 3.4199999999999999, 3.46, 3.5, 3.5164725, 3.54, 3.5800000000000001, 3.6200000000000001, 3.6600000000000001, 3.7000000000000002, 3.7400000000000002, 3.7799999999999998, 3.8199999999999998, 3.846445, 3.8599999999999999, 3.8999999999999999, 3.9399999999999999, 3.98, 4.0199999999999996, 4.0599999999999996, 4.09999999

In [8]:
pair_knots = [2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
              3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
              4.8953, 5.089755, 5.3429525,5.401695,5.4604375]  

mb_knots = [4.5, 4.62, 4.74, 4.86, 4.98, 5.1, 5.22, 5.34, 5.46]
mb_knots

[4.5, 4.62, 4.74, 4.86, 4.98, 5.1, 5.22, 5.34, 5.46]

In [9]:
# cycle over knots of selected models
stats_list = []
for knot in mb_knots:
    edens_knots = [knot]
    stats = downselect(stats_all, pair_knots, edens_knots)
    print(stats['hyperparams'])
    stats_select_md = downselect(stats_md, pair_knots, edens_knots)
    print(stats_select_md['hyperparams'])
    stats['md'] = stats_select_md['md']
    stats_list.append(stats)

{u'pair': [2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [4.5]}
{u'pair': [2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [4.5]}
{u'pair': [2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [4.6200000000000001]}
{u'pair': [2.564897499999

In [10]:
targets['md'] = targets_md['md']

In [11]:
for i, box in enumerate(targets['eos']['box']):
    print(i, box[0,0])

(0, 6.3724472324999999)
(1, 7.0096919557500001)
(2, 7.6469366790000013)
(3, 8.2841814022500007)
(4, 8.9214261255)
(5, 9.5586708487499994)
(6, 10.195915572000001)
(7, 10.833160295250002)
(8, 11.470405018499999)
(9, 12.107649741749999)
(10, 12.744894465)
(11, 13.382139188250001)
(12, 14.0193839115)
(13, 14.656628634749998)
(14, 15.293873357999999)
(15, 15.93111808125)
(16, 16.568362804499998)
(17, 17.205607527750001)
(18, 17.842852251)
(19, 18.480096974249999)
(20, 19.117341697499999)


In [12]:
9.5586708487499994/12.74489446*2.74

2.055000000806205

In [13]:
targets['eos_bcc']['box'] = targets['eos_bcc']['box'][0:5]
targets['eos_bcc']['xyz'] = targets['eos_bcc']['xyz'][0:5]
targets['eos_bcc']['temp'] = targets['eos_bcc']['temp'][0:5]
targets['eos_bcc']['beta'] = targets['eos_bcc']['beta'][0:5]
targets['eos_bcc']['energy'] = targets['eos_bcc']['energy'][0:5]
targets['eos_bcc']['forces'] = targets['eos_bcc']['forces'][0:5]
stats['eos_bcc']['energy'] = stats['eos_bcc']['energy'][0:5]
stats['eos_bcc']['forces'] = stats['eos_bcc']['forces'][0:5]

In [14]:
targets['eos']['box'] = targets['eos']['box'][4:9]
targets['eos']['xyz'] = targets['eos']['xyz'][4:9]
targets['eos']['temp'] = targets['eos']['temp'][4:9]
targets['eos']['beta'] = targets['eos']['beta'][4:9]
targets['eos']['energy'] = targets['eos']['energy'][4:9]
targets['eos']['forces'] = targets['eos']['forces'][4:9]
stats['eos']['energy'] = stats['eos']['energy'][4:9]
stats['eos']['forces'] = stats['eos']['forces'][4:9]

In [15]:
targets['relax']['box'][1] = targets['inf']['box'][0]
targets['relax']['xyz'][1] = targets['inf']['xyz'][0]
targets['relax']['temp'][1] = targets['inf']['temp'][0]
targets['relax']['beta'][1] = targets['inf']['beta'][0]
targets['relax']['energy'][1] = targets['inf']['energy'][0]
targets['relax']['forces'][1] = targets['inf']['forces'][0]
stats['relax']['energy'][1] = stats['inf']['energy'][0]
stats['relax']['forces'][1] = stats['inf']['forces'][0]

In [16]:
targets['relax']['energy']

[-1139.2,
 -2.9688679595213743e-239,
 -1136.4010143821092,
 -1136.7508187321096,
 -1190.2611772847656,
 -1127.1746971578905]

In [17]:
targets['eos_fcc']['weight'] = 0.0
targets['bain']['weight'] = 0.0
targets['eos']['weight'] = 0.0
targets['eos_bcc']['weight'] = 0.0
targets['relax']['weight'] = 0.2
targets['inf']['weight'] = 0.0
#targets['liq_5000K']['weight'] = 0.1

In [18]:
[(key, targets[key]['weight']) for key in targets.keys()]

[('bcc_npt_langevin_3700K', 1.0),
 ('eos', 0.0),
 ('vac_npt_langevin_2000K', 1.0),
 ('md', 1.0),
 ('inf', 0.0),
 ('relax', 0.2),
 ('screw_111_npt_langevin_2000K', 1.0),
 ('bcc_npt_langevin_300K', 1.0),
 ('eos_bcc', 0.0),
 ('liq_5000K', 1.0),
 ('fcc_npt_langevin_300K', 1.0),
 ('vacint', 1.0),
 ('eos_fcc', 0.0),
 ('i110_npt_langevin_2000K', 1.0),
 ('liq_3700K', 1.0),
 ('i111_npt_langevin_2000K', 1.0),
 ('bain', 0.0)]

In [19]:
targets['eos']['temp'] = [5000.0, 5000., 5000., 5000.]
targets['eos']['beta'] = [1/5000.0, 1/5000., 1/5000., 1/5000.]

In [20]:
marinica_params = [-5.946454, -0.049477, 9.60851701e+01,
            -1.84410924e+02,   9.35784080e+01,  -7.98358265e+00,
             7.47034093e+00,  -1.52756044e+00,   1.25205933e+00,
             1.63082162e+00,  -1.41854775e+00,  -8.19936046e-01,
             1.98013514e+00,  -6.96430180e-01,   3.04546910e-02,
            -1.63131143e+00,   1.38409896e+00]
params_fix = marinica_params

In [21]:
#multi_pars = [[-5.946454, -0.049477] for _ in range(len(mb_knots))] 
#multi_pars = [[-9.58668440e-01, 1.74045428e-05] for _ in range(len(mb_knots))] #-1.89968473e+00,   3.88206332e-04
#multi_pars = [[-1.89968473e+00,   3.88206332e-04] for _ in range(len(mb_knots))]
multi_pars = [[-8.81890867e-01,   1.13386735e-05] for _ in range(len(mb_knots))]

In [22]:
multi_pars = [multi_pars[0] + marinica_params[2:]]

In [23]:
params_fix = []
# multi_pars = [[ -7.24625439e-01,  -4.20161230e-06,  -3.07148614e+02,
#          3.59616000e+02,  -1.17075707e+02,  -1.45265859e+01,
#          3.30731395e+01,  -1.81428637e+01,   2.94106177e+00,
#          6.59773245e-02,  -9.02508924e-01,  -4.76781642e-01,
#          1.78928078e+00,  -7.70746632e-01,   1.09292966e-01,
#         -1.66406876e+00,   1.35267922e+00]]

multi_pars = [[ -5.25244991e-01,   2.73463633e-05,  -2.58796319e+02,
         2.94565795e+02,  -8.82445708e+01,  -1.63333425e+01,
         3.45810224e+01,  -2.08622374e+01,   4.72644253e+00,
        -9.96075349e-01,  -7.98992333e-01,  -4.39205593e-01,
         1.93165951e+00,  -8.26849418e-01,   1.03361300e-01,
        -1.70815348e+00,   1.33061825e+00]]

In [24]:
def optimize_EAM_mp(pars, targets, stats, utot_func, params_fix=[]):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_box, None, 0.05, 0, params_fix), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)

    return tuple([output[1], output[0]])

In [25]:
params_output = []

pool = mp.Pool()

best_params = [[0, np.array(multi_pars[0])]]

for ieam, stats in enumerate(reversed(stats_list[:])):

    get_sd = functools.partial(sd2_loss, targets=targets, stats=stats, utot_func=utot_EAM_per_box, ftot_func=None, dl=0.05, verbose=1, params_fix=params_fix)

    get_opt = functools.partial(optimize_EAM_mp, targets=targets, stats=stats, utot_func=utot_EAM_per_box, params_fix=params_fix)

    # initial ordering
    print('Initial params:', best_params[0])
    
    #distances = pool.map(get_sd, multi_pars)
    
    #distances = [get_sd(multi_pars[ieam], params_fix=params_fix)]
    distances = [get_sd(best_params[0][1], params_fix=params_fix)]

    
    optimal_params = zip(distances, [best_params[0][1]])
    #optimal_params = zip(distances, multi_pars[ieam:ieam+1])
    best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
    m_pars = [p[1] for p in best_params]

    print('Best params:', best_params)
    print('ieam:', ieam)
    #m_pars = multi_pars

    for i in range(8):
        #optimal_params = pool.map(get_opt, m_pars)
        optimal_params = [get_opt(m_pars)]
        best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
        m_pars = [p[1] for p in best_params]
        
        print("Iteration {}, best params: {}".format(i, best_params))

    print('Final Best params for knot', stats['hyperparams']['edens'], mb_knots[ieam],':', best_params)
    
    params_output.append(best_params)

pool.close()
pool.join()

('Initial params:', [0, array([ -5.25244991e-01,   2.73463633e-05,  -2.58796319e+02,
         2.94565795e+02,  -8.82445708e+01,  -1.63333425e+01,
         3.45810224e+01,  -2.08622374e+01,   4.72644253e+00,
        -9.96075349e-01,  -7.98992333e-01,  -4.39205593e-01,
         1.93165951e+00,  -8.26849418e-01,   1.03361300e-01,
        -1.70815348e+00,   1.33061825e+00])])
loss 5.87360570426e-06 5.87360570426e-06 0.0
('Best params:', [(5.8736057042605849e-06, array([ -5.25244991e-01,   2.73463633e-05,  -2.58796319e+02,
         2.94565795e+02,  -8.82445708e+01,  -1.63333425e+01,
         3.45810224e+01,  -2.08622374e+01,   4.72644253e+00,
        -9.96075349e-01,  -7.98992333e-01,  -4.39205593e-01,
         1.93165951e+00,  -8.26849418e-01,   1.03361300e-01,
        -1.70815348e+00,   1.33061825e+00]))])
('ieam:', 0)
Iteration 0, best params: [(5.1601168398806099e-06, array([ -4.98375644e-01,   3.01921192e-05,  -2.02098139e+02,
         2.25179959e+02,  -5.94856437e+01,  -2.25513660e+01

loss 0.875278295473 0.875278295473 0.0
('Best params:', [(0.8752782954731203, array([  2.14094059e+00,   8.17607364e-04,   4.40609589e+02,
         1.24243855e+02,  -1.13432077e+03,   1.58677701e+03,
        -1.38538574e+03,   3.95823264e+02,  -6.41988786e-01,
         2.87331115e+01,  -2.52299875e+01,  -4.86350874e-01,
         4.36403725e+00,   3.19560231e+00,  -7.36247827e-01,
        -5.29382927e-01,  -1.72039789e+00]))])
('ieam:', 2)
Iteration 0, best params: [(0.001183308526781584, array([  5.23350918e-01,   6.65281161e-04,   7.79684967e+02,
        -8.61306828e+00,  -1.21357682e+03,   1.60096714e+03,
        -1.34828434e+03,   3.59417078e+02,  -6.73855957e-02,
         3.71505497e+01,  -2.93609464e+01,   6.30382338e-01,
        -6.79934179e+00,   1.53208532e+01,  -2.72531303e-01,
        -2.27966069e+00,  -2.74692145e+00]))]
Iteration 1, best params: [(0.0011253083013677636, array([  1.50274687e+00,   9.23303636e-04,   9.96615792e+02,
        -1.31142717e+02,  -1.25915314e+03,  

('ieam:', 4)
Iteration 0, best params: [(0.001052573030371379, array([ -1.71031966e+00,   3.42730858e-04,   9.23527600e+02,
        -5.14306416e+01,  -1.28406772e+03,   1.64109074e+03,
        -1.45266538e+03,   4.58668374e+02,  -3.97075095e+01,
         6.32560561e+01,  -3.50446076e+01,  -5.46367942e+00,
        -1.85863047e+00,   1.57235341e+01,   3.10008069e-01,
        -3.38169933e+00,  -2.44319426e+00]))]
Iteration 1, best params: [(0.0010519938844900975, array([ -1.70345497e+00,   3.57672230e-04,   1.07196195e+03,
        -2.47314394e+02,  -1.19994102e+03,   1.62751117e+03,
        -1.44323350e+03,   4.54331226e+02,  -3.87410249e+01,
         6.29323075e+01,  -3.50787522e+01,  -5.47244737e+00,
        -1.83078331e+00,   1.57082014e+01,   3.19320728e-01,
        -3.37786298e+00,  -2.45005042e+00]))]
Iteration 2, best params: [(0.0010513414916938748, array([ -1.55830263e+00,   4.23077191e-04,   1.04409579e+03,
        -2.09025286e+02,  -1.21814595e+03,   1.62682689e+03,
        -1.

loss 0.00244741015494 0.00244741015494 0.0
('Best params:', [(0.0024474101549369172, array([ -1.93549909e+00,   4.77578063e-04,   1.08139982e+03,
        -2.69466758e+02,  -1.17795308e+03,   1.60476972e+03,
        -1.40246825e+03,   4.27323334e+02,  -3.11433076e+01,
         5.51856074e+01,  -2.96334990e+01,  -9.46261830e+00,
         6.93870881e-01,   1.54671041e+01,   2.03085601e-01,
        -2.94692707e+00,  -2.77695004e+00]))])
('ieam:', 6)
Iteration 0, best params: [(0.0010272494384674649, array([ -2.42023845e+00,   4.96035107e-04,   1.06042567e+03,
        -2.35729006e+02,  -1.19752179e+03,   1.61040333e+03,
        -1.40756848e+03,   4.30160958e+02,  -3.19357363e+01,
         5.54454378e+01,  -2.94423520e+01,  -9.50227593e+00,
         6.93158417e-01,   1.54439275e+01,   1.97342910e-01,
        -2.91271783e+00,  -2.76684001e+00]))]
Iteration 1, best params: [(0.0010272220378843371, array([ -2.41737386e+00,   4.90281415e-04,   1.06650372e+03,
        -2.49076839e+02,  -1.1882072

loss 0.0030789763567 0.0030789763567 0.0
('Best params:', [(0.0030789763566986981, array([ -1.94654803e+00,   1.13380393e-03,   9.78768519e+02,
        -1.68497838e+02,  -1.17361973e+03,   1.62003161e+03,
        -1.49862578e+03,   5.00693130e+02,  -3.44260980e+01,
         4.34477856e+01,  -1.86954678e+01,  -2.32738702e+01,
         2.27902925e+01,   2.51106280e+00,  -3.90237208e-01,
        -2.86385534e+00,  -6.80520391e-01]))])
('ieam:', 8)
Iteration 0, best params: [(0.0008924174841215695, array([ -2.79058162e+00,   8.15363751e-04,   9.52209816e+02,
        -1.45591045e+02,  -1.17655825e+03,   1.61870738e+03,
        -1.49939711e+03,   5.01191866e+02,  -3.43300373e+01,
         4.33866026e+01,  -1.83911091e+01,  -2.32932969e+01,
         2.27504240e+01,   2.66887854e+00,  -4.26120459e-01,
        -2.76007168e+00,  -7.89086279e-01]))]
Iteration 1, best params: [(0.00089232097251562017, array([ -2.73859678e+00,   9.31007622e-04,   9.81327655e+02,
        -1.75490584e+02,  -1.16848405

In [50]:
pool.close()
pool.join()

In [30]:


params_output = [[-4.20354715e+00,  -2.33102863e-03,
                 2.78559547e-02, -3.69797280e+03,   9.28651669e+01,
                  -15.2604631,8.88626493,10.7120352,46.1310765,6.70641338,-7.14000617,-2.02174483,
   1.9897676,-1.35491449,-0.812592118,1.99278451,-0.484145757,-0.0478673818,-1.46201396,1.19810233,-0.00736954413,
   -0.00388205423,-0.00512566656,
-3.08290042e-02,-3.42747549e+03,
                  85.8053818,31.7983469,-1.52861585,-1.77508182,0.207405945
                ]]

# [ -4.20354715e+00,  -2.33102863e-03,   2.78559547e-02, -3.69797280e+03,   9.28651669e+01,  -3.08290042e-02,
#         -3.42747549e+03]



print(params_output)

[[-4.20354715, -0.00233102863, 0.0278559547, -3697.9728, 92.8651669, -15.2604631, 8.88626493, 10.7120352, 46.1310765, 6.70641338, -7.14000617, -2.02174483, 1.9897676, -1.35491449, -0.812592118, 1.99278451, -0.484145757, -0.0478673818, -1.46201396, 1.19810233, -0.00736954413, -0.00388205423, -0.00512566656, -0.0308290042, -3427.47549, 85.8053818, 31.7983469, -1.52861585, -1.77508182, 0.207405945]]


In [30]:
# convert lists of optimal parameters to dictionaries
params_dicts = []
for params in params_output:
    params_dicts.append(to_param_dict(params, stats['hyperparams']))

In [31]:
params_dicts

[{'hyperparams': {u'edens': [2.1000000000000001,
    2.2200000000000002,
    2.5,
    2.8663175000000001,
    3.1000000000000001,
    3.5,
    4.6200000000000001],
   u'pair': [2.1000000000000001,
    2.2200000000000002,
    2.46,
    2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003,
    5.54,
    5.5800000000000001,
    5.6200000000000001]},
  'params': {'edens': [-0.0308290042,
    -3427.47549,
    85.8053818,
    31.7983469,
    -1.52861585,
    -1.77508182,
    0.207405945],
   'embed': [-4.20354715, -0.00233102863],
   'lrcorr': [],
   'pair': [0.0278559547,
    -3697.9728,
    92.8651669,
    -15.2604631,
    8.88626493,
    10.7120352,
    46.1310765,
    6.70641338,
    -7.14000617,
   

In [32]:
file_name = os.path.join(working, "params_out.pickle")

for params in params_dicts:
    params_to_pickle(params, file_name)

In [33]:
with open(os.path.join(working, 'params_out' + '.pickle'), 'rb') as fi:
    retreived_params = pickle.load(fi)  

In [34]:
retreived_params

[{'hyperparams': {u'edens': [2.5,
    2.8663175000000001,
    3.1000000000000001,
    3.5,
    4.5],
   u'pair': [2.2200000000000002,
    2.46,
    2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': [106.853821,
    27.4234479,
    -1.26300179,
    -1.66378666,
    0.224848394],
   'embed': [-4.65804603, -0.00756457588],
   'lrcorr': [],
   'pair': [-168.77752,
    156.052349,
    14.2649908,
    -4.55941009,
    19.761334,
    48.4204936,
    6.4946677,
    -6.62233324,
    -2.16547494,
    1.30932417,
    -0.83328148,
    -0.832333597,
    2.09417158,
    -0.619566076,
    -0.0953112547,
    -1.49756931,
    1.2716794]}},
 {'hyperparams': {u'edens': [2.2200000000000002,

In [35]:
retreived_params[-2:]

[{'hyperparams': {u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004],
   u'pair': [2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': [14.2447861, 0.868030511, 0.50452072, 0.00877385016],
   'embed': [-5.9173299, -0.00439326285],
   'lrcorr': [],
   'pair': [-48.6300768,
    122.549311,
    -60.4621784,
    9.09485341,
    -0.79271282,
    -0.309711402,
    4.12680657,
    0.540819962,
    -1.23317275,
    -0.795900416,
    2.17660195,
    -0.639345527,
    -0.00230178698,
    -1.7138732,
    1.32277953]}},
 {'hyperparams': {u'edens': [2.1000000000000001,
    2.2200000000000002,
    2.5,
    2.8663175000000001,
    3.1000000000000001,
    3.5,
    4.620000000000